In [1]:
import pymongo

In [2]:
#create connection
connection_string = 'enter connection string'

#connection to mongoDB atlas
client = pymongo.MongoClient(connection_string)

In [3]:
#creating database
db = client['sample_airbnb']

#collection - similar to table
coll = db['listingsAndReviews']

In [4]:
#1
results = [
    {
        "$group": {
            "_id": "$address.country",
            "average_price": {"$avg": "$price"}
        }
    },
    {
        "$sort": {"average_price": -1}
    }
]

result = coll.aggregate(results)

for doc in result:
    print(doc)

{'_id': 'Hong Kong', 'average_price': Decimal128('773.7866666666666666666666666666667')}
{'_id': 'Brazil', 'average_price': Decimal128('525.3465346534653465346534653465347')}
{'_id': 'China', 'average_price': Decimal128('405.3684210526315789473684210526316')}
{'_id': 'Turkey', 'average_price': Decimal128('368.1467473524962178517397881996974')}
{'_id': 'Australia', 'average_price': Decimal128('197.8065573770491803278688524590164')}
{'_id': 'United States', 'average_price': Decimal128('185.7692307692307692307692307692308')}
{'_id': 'Spain', 'average_price': Decimal128('100.8262243285939968404423380726698')}
{'_id': 'Canada', 'average_price': Decimal128('100.1171032357473035439137134052388')}
{'_id': 'Portugal', 'average_price': Decimal128('69.18198198198198198198198198198198')}


In [5]:
#2
results = [
    {
        "$match": {
            "bedrooms": {"$gt": 3} 
        }
    },
    {
        "$group": {
            "_id": "$address.country", 
            "listings": {"$sum": 1} 
        }
    },
    {
        "$sort": {"listings": 1} 
    }
]

result = coll.aggregate(results)

for doc in result:
    print(doc)

{'_id': 'Hong Kong', 'listings': 3}
{'_id': 'Turkey', 'listings': 16}
{'_id': 'Canada', 'listings': 24}
{'_id': 'Portugal', 'listings': 29}
{'_id': 'Brazil', 'listings': 30}
{'_id': 'Spain', 'listings': 37}
{'_id': 'United States', 'listings': 42}
{'_id': 'Australia', 'listings': 48}


In [6]:
#3
results = [
    {
        "$group": {
            "_id": {
                "propertyType": "$property_type", 
                "hostId": "$host.host_id",          
                "hostName": "$host.host_name"       
            },
            "numListings": {"$sum": 1}     
        }
    },
    {
        "$sort": {"numListings": -1}         
    },
    {
        "$group": {
            "_id": "$_id.propertyType",
            "topHosts": {
                "$push": {
                    "hostId": "$_id.hostId",
                    "hostName": "$_id.hostName",
                    "numListings": "$numListings"
                }
            }
        }
    },
    {
        "$project": {
            "topHosts": {"$slice": ["$topHosts", 5]}
        }
    }
]

result = coll.aggregate(results)

for doc in result:
    print({doc['_id']})
    for host in doc['topHosts']:
        print(f'{host['hostId']}, {host['hostName']}, {host['numListings']}')
    print()

{'Heritage hotel (India)'}
146227207, Best Western Plus Hotel Hong Kong, 1

{'Bed and breakfast'}
10790369, Ewa, 2
11369496, Brandyn, 2
27917181, Baran, 1
39815965, Raquel, 1
2657251, Darcy And Tim, 1

{'Bungalow'}
3383921, Jill, 1
53507053, Bobby, 1
164714644, Jian, 1
25102990, Tomas And Julia, 1
41587571, Debbie, 1

{'Hostel'}
159283889, Eylül, 3
128381201, Yuki, 2
7104890, Engin, 1
120225790, Katherine, 1
27773646, Kenneth, 1

{'Houseboat'}
51033251, May, 1

{'Guest suite'}
99254174, Simone, 2
189860862, Cherry, 2
1771300, Jump In Bed, 2
142642175, Philip, 1
100464984, Sam, 1

{'Serviced apartment'}
38459934, Alejandro, 9
122131447, Debe, 4
16922131, YourOpo, 4
36133410, Patty And Beckett, 4
122408520, Royal, 3

{'Camper/RV'}
36714324, David, 1
22091788, Graça, 1

{'Loft'}
89588787, Porto, 2
171236207, Apartment Barcelona, 2
19059657, Yanick, 2
1138920, Miguel & João, 2
21494266, Antonio, 1

{'Campsite'}
83224497, Jason, 1

{'Train'}
18766862, Margaret, 1

{'Castle'}
3548411, CASA H

In [7]:
#4
results = [
    {
        "$match": {
            "review_scores.review_scores_cleanliness": { "$exists": True } 
        }
    },
    {
        "$group": {
            "_id": "$address.country",  
            "averageCleanlinessScore": { "$avg": "$review_scores.review_scores_cleanliness" } 
        }
    },
    {
        "$sort": { "averageCleanlinessScore": -1 } 
    },
    {
        "$limit": 5 
    }
]

result = coll.aggregate(results)

for doc in result:
    print(f'{doc['_id']}, {doc['averageCleanlinessScore']:.2f}')

Portugal, 9.54
Brazil, 9.50
United States, 9.42
Canada, 9.27
Australia, 9.26


In [8]:
#5
results = [
    {
        "$group": {
            "_id": "$host.host_id",  
            "hostName": { "$first": "$host.host_name" }, 
            "numBookings": { "$sum": 1 }  
        }
    },
    {
        "$sort": { "numBookings": -1 }   
    }
]

result = coll.aggregate(results)

for doc in result:
    print(f'{doc['_id']}, {doc['hostName']}, {doc['numBookings']}')

97240131, Jov, 18
12243051, Sonder, 11
4459553, Eva&Jacques, 9
38459934, Alejandro, 9
30907275, Liiiving, 7
3953109, Feels Like Home, 7
69552498, Captain Cook Resorts, 6
122131447, Debe, 6
74079684, Marina, 6
43793840, Mark, 6
51017586, Rentals Maui, 5
14861546, Tane Residence, 5
21873399, Sol, 5
387422, Simplissimmo, 5
4255118, Ricardo, 5
67709885, Morgan, 5
30098437, Dragon Hostel, 5
16922131, YourOpo, 5
170716140, Robbie & Kay, 5
171236207, Apartment Barcelona, 4
582060, Fernando, 4
6504006, Chile, 4
4584648, Ivy, 4
299462, Stay U-Nique, 4
12730028, Host Wise, 4
318040, Mike, 4
42618840, Great Vacation Retreats, 4
10496350, Elite, 4
2686177, João, 4
41842769, Resortica, 4
2154262, Condominium Rentals Hawaii, 4
2349948, Hawaii Dream Realty TEAM, 4
756093, Joshua, 4
36133410, Patty And Beckett, 4
1138920, Miguel & João, 4
5786717, Viktoria, 4
21792167, Madlen, 3
179663253, Chrystel, 3
76545056, Rita, 3
153745488, José, 3
34386367, Kauai Calls!, 3
102947901, Claudia, 3
52473150, Danny,

In [9]:
#6
results = [
    {
        "$match": {
            "bedrooms": { "$gt": 3 },              
            "number_of_reviews": { "$gte": 10 }    
        }
    },
    {
        "$group": {
            "_id": "$address.country",   
            "averagePrice": { "$avg": "$price" }  
        }
    },
    {
        "$sort": { "averagePrice": -1 } 
    }
]

result = coll.aggregate(results)

for doc in result:
    print(f'{doc['_id']} {doc['averagePrice']}')

Brazil 3253.00
Turkey 501.00
United States 415.0370370370370370370370370370370
Australia 409.6363636363636363636363636363636
Spain 239.1875
Portugal 174.8235294117647058823529411764706
Canada 144.00


In [10]:
#7
results = [
    {
        "$match": {
            "review_scores.review_scores_communication": { "$exists": True }, 
            "number_of_reviews": { "$gte": 10 } 
        }
    },
    {
        "$sort": { "review_scores.communication": -1 } 
    },
    {
        "$limit": 10 
    }
]

result = coll.aggregate(results)

for doc in result:
    print(f'{doc['name']}')

Sydney Hyde Park City Apartment (checkin from 6am)
A Casa Alegre é um apartamento T1.
Be Happy in Porto
Cozy house at Beyoğlu
City center private room with bed
Copacabana Apartment Posto 6
Ocean View Waikiki Marina w/prkg
Ribeira Charming Duplex
New York City - Upper West Side Apt
Surry Hills Studio - Your Perfect Base in Sydney


In [11]:
#8
results = [
    {
        "$unwind": "$amenities" 
    },
    {
        "$group": {
            "_id": "$amenities", 
            "count": { "$sum": 1 } 
        }
    },
    {
        "$sort": { "count": -1 } 
    },
    {
        "$limit": 5 
    }
]

result = coll.aggregate(results)

for doc in result:
    print(f'{doc['_id']}')

Wifi
Essentials
Kitchen
TV
Hangers


In [12]:
#9
results = [
    {
        "$group": {
            "_id": "$property_type", 
            "count": { "$sum": 1 } 
        }
    },
    {
        "$sort": { "count": -1 } 
    }
]

result = coll.aggregate(results)

for doc in result:
    print(f'{doc['_id']} {doc['count']}')

Apartment 3626
House 606
Condominium 399
Serviced apartment 185
Loft 142
Townhouse 108
Guest suite 81
Bed and breakfast 69
Boutique hotel 53
Guesthouse 50
Hostel 34
Villa 32
Hotel 26
Aparthotel 23
Cottage 20
Other 18
Cabin 15
Bungalow 14
Resort 11
Casa particular (Cuba) 9
Farm stay 9
Tiny house 7
Nature lodge 2
Camper/RV 2
Chalet 2
Boat 2
Campsite 1
Barn 1
Earth house 1
Hut 1
Castle 1
Pension (South Korea) 1
Train 1
Heritage hotel (India) 1
Houseboat 1
Treehouse 1


In [13]:
#10
results = [
    {
        "$group": {
            "_id": "$host.host_name",  
            "totalRevenue": { "$sum": "$price" } 
        }
    },
    {
        "$sort": { "totalRevenue": -1 } 
    },
    {
        "$limit": 5 
    }
]

result = coll.aggregate(results)

for doc in result:
    print(f"{doc['_id']}, ${doc['totalRevenue']}")

Ipek, $48842.00
Debe, $14852.00
Pllumbaj, $11681.00
Giovana, $11190.00
Fernando, $10410.00


In [14]:
#11
results = [
    {
        "$project": {
            "host_name": "$host.host_name", 
            "minStayPrice": {
                "$add": [
                    { "$multiply": ["$price", {'$toInt': "$minimum_nights"}]},
                    "$cleaning_fee"  
                ]
            }
        }
    },
    {
        "$group": {
            "_id": "$host_name", 
            "maxMinStayPrice": { "$max": "$minStayPrice" } 
        }
    },
    {
        "$sort": { "maxMinStayPrice": -1 } 
    },
    {
        "$limit": 5 
    }
]

result = coll.aggregate(results)

for doc in result:
    print(f"{doc['_id']}, ${doc['maxMinStayPrice']}")

Sinem, $474744.00
Genevieve, $225060.00
Claudia, $112348.00
Karen, $102470.00
Melina, $87435.00
